In [1]:
%matplotlib inline
import numpy as np
import scipy
import pandas
import matplotlib.pyplot as plt
#import statsmodels.formula.api as sm
import seaborn as sns
import sklearn as sl
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
pandas.set_option('display.max_columns', 20)
pandas.set_option('display.width', 350)

//miniconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data=pandas.read_json('./dataset/song.json')

In [3]:
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [4]:
data.shape

(4000, 6)

In [5]:
data.dtypes

id                   object
user_id               int64
user_state           object
user_sign_up_date    object
song_played          object
time_played          object
dtype: object

In [6]:
#fixing dates
data['time_played']=  pandas.to_datetime(data['time_played'], format='%Y-%m-%d %H:%M:%S')
data['user_sign_up_date'] = pandas.to_datetime(data['user_sign_up_date'], format='%Y-%m-%d')

In [7]:
data.dtypes

id                           object
user_id                       int64
user_state                   object
user_sign_up_date    datetime64[ns]
song_played                  object
time_played          datetime64[ns]
dtype: object

In [8]:
data['song_played'] = data['song_played'].str.lower()

In [10]:
#From how many states we have at least one user?
print("For", 50 - data['user_state'].nunique(), "states we have no users")

For 9 states we have no users


In [11]:
data_state_count = data.groupby('user_state').user_id.nunique()

In [12]:
print(data_state_count.nlargest(3, keep='all'))

user_state
New York      23
California    21
Texas         15
Name: user_id, dtype: int64


In [13]:
print(data_state_count.nsmallest(3, keep='all'))

user_state
Arizona         1
Connecticut     1
Idaho           1
Iowa            1
Kansas          1
Nebraska        1
New Mexico      1
North Dakota    1
Rhode Island    1
Name: user_id, dtype: int64


In [15]:
all_states = [
         'Alabama','Alaska','Arizona','Arkansas','California','Colorado',
         'Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho', 
         'Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
         'Maine', 'Maryland','Massachusetts','Michigan','Minnesota',
         'Mississippi', 'Missouri','Montana','Nebraska','Nevada',
         'New Hampshire','New Jersey','New Mexico','New York',
         'North Carolina','North Dakota','Ohio',    
         'Oklahoma','Oregon','Pennsylvania','Rhode Island',
         'South Carolina','South Dakota','Tennessee','Texas','Utah',
         'Vermont','Virginia','Washington','West Virginia',
         'Wisconsin','Wyoming'
    ]

In [16]:
print(len(np.intersect1d(data['user_state'].unique(), all_states))==data['user_state'].nunique())

True


In [17]:
print("The following", len(np.setxor1d(data['user_state'].unique(), all_states))  ,"states have no users: ", np.setxor1d(data['user_state'].unique(), all_states))

The following 9 states have no users:  ['Delaware' 'Hawaii' 'Maine' 'Montana' 'Nevada' 'New Hampshire'
 'South Dakota' 'Vermont' 'Wyoming']


In [18]:
data_engagement = data.groupby(['user_id', 'user_state']).size().reset_index(name='songs_user')

In [19]:
data_engagement = data_engagement.groupby(['user_state']).apply(
                    lambda x: pandas.Series({
                             # avg songs
                            'engagement': x['songs_user'].mean(),
                             # count unique users 
                            'count_unique_users': x['user_id'].nunique()
  })
).reset_index()

In [20]:
data_engagement = data_engagement.loc[data_engagement['count_unique_users'] > np.median(data_engagement.count_unique_users)]

In [21]:
data_engagement.head(5)

,user_state,engagement,count_unique_users
0,Alabama,26.000000,4.0
4,California,20.238095,21.0
7,Florida,25.714286,7.0
8,Georgia,22.500000,6.0
10,Illinois,21.285714,7.0


In [22]:
#Top 3 states
print("Top 3 states for user engagement are: \n", data_engagement.nlargest(3, columns = 'engagement', keep='all'))

Top 3 states for user engagement are: 
         user_state  engagement  count_unique_users
0          Alabama   26.000000                 4.0
7          Florida   25.714286                 7.0
26  North Carolina   25.666667                 6.0


In [23]:
#Bottom 3 
print("Bottom 3 states for user engagement are: \n", data_engagement.nsmallest(3, columns = 'engagement', keep='all'))

Bottom 3 states for user engagement are: 
        user_state  engagement  count_unique_users
19      Minnesota   10.500000                 4.0
11        Indiana   13.750000                 4.0
17  Massachusetts   15.166667                 6.0


# Q: The CEO wants to send a gift to the first user who signed-up for each state. That is, the first user who signed-up from California, from Oregon, etc. Create a list of those users.

In [24]:
idx = data[["user_state", "user_id", "user_sign_up_date"]].groupby('user_state')['user_sign_up_date'].transform(min) == data['user_sign_up_date']

In [25]:
data_min = data[idx][["user_state", "user_id", "user_sign_up_date"]].drop_duplicates()

In [26]:
print(data_min.query("user_state == \"New York\""))

    user_state  user_id user_sign_up_date
52    New York       19        2015-05-02
60    New York       22        2015-05-02
196   New York       27        2015-05-02
228   New York       12        2015-05-02
266   New York       10        2015-05-02


# Q: Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next

In [27]:
#Firstly let's create a dataset where we add a column next_song which is the song played right after by the same user. 
#If there are more than 30 minutes between consecutive songs, we consider that a new session and therefore it is not counted as next_song. 30 minutes of inactivity is standard for defining a new session. If a song is the last of the session, we remove it from the dataset
  
#only keep variables we care about and sort by user_id and time_played
data_MC = data[['song_played', 'time_played', 'user_id']].sort_values(by=['user_id', 'time_played'])

In [28]:
print(data_MC.query("user_id==1"))

                       song_played         time_played  user_id
1952                     yesterday 2015-06-05 14:30:22        1
619   while my guitar gently weeps 2015-06-07 18:54:56        1
2719     the long and winding road 2015-06-08 22:37:41        1
669      reprise / day in the life 2015-06-10 18:00:05        1
353                    i feel fine 2015-06-15 15:46:46        1
993                  hello goodbye 2015-06-19 14:54:57        1
627             here comes the sun 2015-06-21 21:53:48        1
1373             can't buy me love 2015-06-22 08:05:01        1
1563                      birthday 2015-06-25 12:32:22        1
3172            here comes the sun 2015-06-25 20:28:47        1


In [30]:
#add next song and corresponding timestamp
data_MC['next_song'] = data_MC.groupby('user_id')['song_played'].shift(-1)
data_MC['next_time_played'] = data_MC.groupby('user_id')['time_played'].shift(-1)

In [33]:
from datetime import datetime

In [35]:
#remove last song for each session. This happens when next_time_played is NA or when next_time_played - time_played > 30 mins
data_MC = data_MC.dropna(subset = ['next_time_played'])

In [36]:
data_MC = data_MC[(data_MC.next_time_played-data_MC.time_played)/pandas.Timedelta(minutes=1) < 30]

In [37]:
#also remove when next song is same as current song. Would not make any sense as suggestion
data_MC = data_MC[data_MC.song_played != data_MC.next_song]
  
#let's estimate for each song, the next song with the highest count
data_MC = data_MC.groupby(['song_played', 'next_song']).size().reset_index(name = 'count')
idx = data_MC.groupby(['song_played'])['count'].transform(max) == data_MC['count']
data_MC = data_MC[idx] 
print(data_MC[['song_played', 'next_song']].head())

         song_played                 next_song
0  a day in the life           i am the walrus
1  a day in the life  i saw her standing there
2  a day in the life     i've just seen a face
3  a day in the life        ob-la-di, ob-la-da
4  a day in the life                oh darling


In [38]:
#write the function 
def MC_function(song):
  tmp = data_MC [data_MC.song_played == song]
  #if we have no suggestions, pick a song randomly, using as a probability of picking the relative frequency in the original dataset
  if tmp.shape[0]==0:
    return(print("Suggested song after", song, "is:", data[data.song_played != song].song_played.sample(1).values))
  #if we only have one suggestion, simply suggest that one
  if tmp.shape[0]==1:
    return(print("Suggested song after", song, "is:", tmp.next_song.values))
  #if we have more than one suggestion, all equally likely, randomly pick one
  if tmp.shape[0]>1:
    return(print("Suggested song after", song, "is:", tmp.next_song.sample(1).values))


In [39]:
print(data_MC[data_MC.song_played == "eight days a week"])

Empty DataFrame
Columns: [song_played, next_song, count]
Index: []


In [40]:
MC_function("eight days a week")

Suggested song after eight days a week is: ['the long and winding road']


In [41]:
#pick a song for which we have one most likely next song
data_MC[data_MC.song_played == "bad boy"]

,song_played,next_song,count
13,bad boy,don't let me down,1


In [42]:
MC_function("bad boy")

Suggested song after bad boy is: ["don't let me down"]


In [43]:
#pick a song for which we have multiple most likely next songs
data_MC[data_MC.song_played == "eleanor rigby"]

,song_played,next_song,count
28,eleanor rigby,back in the u.s.s.r.,1
29,eleanor rigby,come together,1
30,eleanor rigby,here comes the sun,1
31,eleanor rigby,lucy in the sky with diamonds,1


In [44]:
MC_function("eleanor rigby")

Suggested song after eleanor rigby is: ['here comes the sun']
